In [1]:
!pip install mysql-connector-python pandas


   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.1 MB 1.7 MB/s eta 0:00:10
   ---------------------------------------- 0.2/16.1 MB 1.6 MB/s eta 0:00:10
    --------------------------------------- 0.4/16.1 MB 2.5 MB/s eta 0:00:07
   - -------------------------------------- 0.6/16.1 MB 2.9 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/16.1 MB 4.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.8/16.1 MB 6.1 MB/s eta 0:00:03
   ------ --------------------------------- 2.5/16.1 MB 7.7 MB/s eta 0:00:02
   -------- ------------------------------- 3.5/16.1 MB 8.9 MB/s eta 0:00:02
   ---------- ----------------------------- 4.3/16.1 MB 10.1 MB/s eta 0:00:02
   ------------ --------------------------- 5.0/16.1 MB 10.7 MB/s eta 0:00:02
   -------------- ------------------------- 5.8/16.1 MB 11.2 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.9 kB ? eta -:--:--
   ----------- ---------------------------- 71.7/250.9 kB 1.3 MB/s eta 0:00:01
   ----------------------------------- ---- 225.3/250.9 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import mysql.connector

# Load data from the Excel files
efficiency_file = 'C://luis//ODN//data//EfficiencyCT//EfficiencyCT//EFFCT_24-W43.xlsx'
ftt_file = 'C://luis//ODN//data//FTT//FTT//FTT_24-W43.xlsx'

efficiency_data = pd.read_excel(efficiency_file)
ftt_data = pd.read_excel(ftt_file)

# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="nirvana123",
    database="odn_db"
)
cursor = conn.cursor()



In [3]:

# Insert data into Plants table
def insert_plants(data):
    plants = data[['Plant', 'Area']].drop_duplicates()
    for _, row in plants.iterrows():
        cursor.execute("""
            INSERT INTO Plants (PlantName, Area)
            VALUES (%s, %s)
        """, (row['Plant'], row['Area']))
    conn.commit()



# Populate the database
insert_plants(efficiency_data)


print("Data successfully inserted into the database.")


Data successfully inserted into the database.


In [12]:
def insert_lines(efficiency_data, ftt_data):
    # Combinar nombres de líneas de ambos archivos y eliminar duplicados
    lines_efficiency = efficiency_data[['Plant', 'Line Name']].drop_duplicates()
    lines_ftt = ftt_data[['Plant Name', 'LineName']].rename(columns={'Plant Name': 'Plant', 'LineName': 'Line Name'}).drop_duplicates()
    combined_lines = pd.concat([lines_efficiency, lines_ftt]).drop_duplicates()

    for _, row in combined_lines.iterrows():
        cursor.execute("""
            SELECT PlantID FROM Plants WHERE PlantName = %s
        """, (row['Plant'],))
        result = cursor.fetchone()
        if result is None:
            raise ValueError(f"Plant {row['Plant']} not found in Plants table.")
        
        plant_id = result[0]
        
        # Insertar solo si la línea no existe ya en la tabla
        cursor.execute("""
            SELECT LineID FROM `Lines` WHERE LineName = %s
        """, (row['Line Name'],))
        if cursor.fetchone() is None:
            cursor.execute("""
                INSERT INTO `Lines` (PlantID, LineName)
                VALUES (%s, %s)
            """, (plant_id, row['Line Name']))
    conn.commit()


In [13]:
# Populate the database
insert_plants(ftt_data)


print("Data successfully inserted into the database.")

Data successfully inserted into the database.


In [4]:
def insert_lines(data):
    lines = data[['Plant', 'Line Name']].drop_duplicates()
    for _, row in lines.iterrows():
        cursor.execute("""
            SELECT PlantID FROM Plants WHERE PlantName = %s
        """, (row['Plant'],))
        
        # Consumir todos los resultados de la consulta SELECT
        result = cursor.fetchall()
        if len(result) == 0:
            raise ValueError(f"Plant {row['Plant']} not found in Plants table.")
        
        plant_id = result[0][0]  # Obtener el primer resultado
        
        # Ejecutar la consulta INSERT
        cursor.execute("""
            INSERT INTO `Lines` (PlantID, LineName)
            VALUES (%s, %s)
        """, (plant_id, row['Line Name']))
    conn.commit()


insert_lines(efficiency_data)


print("Data successfully inserted into the database.")

Data successfully inserted into the database.


In [5]:
def insert_efficiency_records(data):
    for _, row in data.iterrows():
        # Consumir los resultados correctamente
        cursor.execute("""
            SELECT LineID FROM `Lines` WHERE LineName = %s
        """, (row['Line Name'],))
        
        # Consumir todos los resultados para evitar conflictos
        results = cursor.fetchall()
        if len(results) == 0:
            raise ValueError(f"Line {row['Line Name']} not found in Lines table.")
        
        line_id = results[0][0]  # Tomar el primer resultado
        
        cursor.execute("""
            INSERT INTO EfficiencyRecords (LineID, Year, Month, Day, Hour, EffPercentage, NumSerials, SerialsUnderTarget, SerialsOverTarget, AvgActualCT, AvgTargetCT)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            line_id, row['Year'], row['Month'], row['Day'], row['Hour'], row['Eff%'], row['#Serials'],
            row['#Serials\n<=Target CT'], row['#Serials\n>Target CT'], row['Avg \nAct CT'], row['Avg \nTarget CT']
        ))
    conn.commit()

insert_efficiency_records(efficiency_data)


print("Data successfully inserted into the database.")


Data successfully inserted into the database.


In [25]:
cursor.close()

True

In [26]:
import pandas as pd
import mysql.connector

# Load data from the Excel files
efficiency_file = 'C://luis//ODN//data//EfficiencyCT//EfficiencyCT//EFFCT_24-W43.xlsx'
ftt_file = 'C://luis//ODN//data//FTT//FTT//FTT_24-W43.xlsx'

efficiency_data = pd.read_excel(efficiency_file)
ftt_data = pd.read_excel(ftt_file)

# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="nirvana123",
    database="odn_db"
)
cursor = conn.cursor(buffered=True)



In [27]:
def insert_ftt_records(data):
    for _, row in data.iterrows():
        # Verificar si la planta existe, si no, crearla
        cursor.execute("""
            SELECT PlantID FROM Plants WHERE PlantName = %s
        """, (row['Plant Name'],))
        
        # Consumir resultados correctamente
        plant_result = cursor.fetchall()
        if len(plant_result) == 0:
            cursor.execute("""
                INSERT INTO Plants (PlantName, Area)
                VALUES (%s, %s)
            """, (row['Plant Name'], 'Default Area'))
            conn.commit()
            cursor.execute("SELECT PlantID FROM Plants WHERE PlantName = %s", (row['Plant Name'],))
            plant_result = cursor.fetchall()
        
        plant_id = plant_result[0][0]

        # Verificar si la línea existe, si no, crearla
        cursor.execute("""
            SELECT LineID FROM `Lines` WHERE LineName = %s
        """, (row['LineName'],))
        
        # Consumir resultados correctamente
        line_result = cursor.fetchall()
        if len(line_result) == 0:
            cursor.execute("""
                INSERT INTO `Lines` (PlantID, LineName)
                VALUES (%s, %s)
            """, (plant_id, row['LineName']))
            conn.commit()
            cursor.execute("SELECT LineID FROM `Lines` WHERE LineName = %s", (row['LineName'],))
            line_result = cursor.fetchall()
        
        line_id = line_result[0][0]

        # Insertar el registro en FTTRecords
        cursor.execute("""
            INSERT INTO FTTRecords (LineID, MoveOutDateTime, FTTEffectiveness)
            VALUES (%s, %s, %s)
        """, (line_id, row['MoveOutDateAndHour'], row['Effectiveness (FTT)']))
    conn.commit()



In [28]:
insert_ftt_records(ftt_data)

In [17]:
# Extraer líneas en ftt_data que no están en la tabla Lines
missing_lines = ftt_data[['Plant Name', 'LineName']].rename(columns={'Plant Name': 'Plant'}).drop_duplicates()

# Buscar cuáles faltan en la base de datos
missing_lines_in_db = []
for _, row in missing_lines.iterrows():
    cursor.execute("""
        SELECT LineID FROM `Lines` WHERE LineName = %s
    """, (row['LineName'],))
    if cursor.fetchone() is None:
        missing_lines_in_db.append(row)

# Mostrar las líneas faltantes
if missing_lines_in_db:
    print("Líneas faltantes en la base de datos:")
    for line in missing_lines_in_db:
        print(line)
else:
    print("Todas las líneas están presentes en la base de datos.")


InternalError: Unread result found